## Get feature values

Make sure that the first data sync for each featureview has finished before requesting feature values.

In [1]:
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient,
    FeatureOnlineStoreServiceClient,
    NearestNeighborQuery,
    SearchNearestEntitiesRequest,
)
from google.cloud.aiplatform_v1beta1.types import (
    feature_online_store_service as feature_online_store_service_pb2,
)

PROJECT_ID = "vertex-feature-store"
PROJECT_NUMBER = "208794590551"
REGION = "europe-west4"
FEATUREONLINESTORE_NAME = "example_online_store"
FEATUREONLINESTORE = f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATUREONLINESTORE_NAME}"
FEATUREVIEW_BASE = f"{FEATUREONLINESTORE}/featureViews/"
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [2]:
admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)
PUBLIC_ENDPOINT = admin_client.get_feature_online_store(
    name=FEATUREONLINESTORE
).dedicated_serving_endpoint.public_endpoint_domain_name

### Retrieve feature values by entity ID

In [5]:
def get_feature_values_by_id(feature_view: str, entitiy_id: str):
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": API_ENDPOINT}
    )
    return data_client.fetch_feature_values(
        feature_online_store_service_pb2.FetchFeatureValuesRequest(
            id=entitiy_id,
            feature_view=feature_view,
        )
    )

In [9]:
feature_view_name = "user_featureview"
entity_id = "1"

response = get_feature_values_by_id(f"{FEATUREVIEW_BASE}{feature_view_name}", entity_id)
print(response)

key_values {
  features {
    value {
      int64_value: 1720101661509420
    }
    name: "feature_timestamp"
  }
  features {
    value {
      string_value: "user1"
    }
    name: "username"
  }
  features {
    value {
      string_value: "user1@gmail.com"
    }
    name: "email"
  }
  features {
    value {
      int64_value: 25
    }
    name: "age"
  }
  features {
    value {
      string_value: "M"
    }
    name: "gender"
  }
}



In [10]:
feature_view_name = "user_rating_featureview"
entity_id = "5"

response = get_feature_values_by_id(f"{FEATUREVIEW_BASE}{feature_view_name}", entity_id)
print(response)

key_values {
  features {
    value {
      int64_value: 1720101661357946
    }
    name: "feature_timestamp"
  }
  features {
    value {
      double_value: 4
    }
    name: "avg_rating_90d"
  }
  features {
    value {
      int64_value: 1
    }
    name: "num_rating_90d"
  }
}



### Search with embedding nearest neighbor

By entity ID


In [11]:
def get_neighbors_by_id(feature_view: str, entitiy_id: str, k: int):
    # NOTE: the client uses the public endpoint here
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": PUBLIC_ENDPOINT}
    )

    query = NearestNeighborQuery(entity_id=entitiy_id, neighbor_count=k)

    request = SearchNearestEntitiesRequest(feature_view=feature_view, query=query)
    return data_client.search_nearest_entities(request=request)

In [12]:
feature_view_name = "movie_embedding_featureview"
feature_view = f"{FEATUREVIEW_BASE}{feature_view_name}"
embedding_dim = 1536

neighbors = get_neighbors_by_id(feature_view, "1", k=3)
print(neighbors)

ServiceUnavailable: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:35.190.19.65:443: Socket closed

By embedding

In [41]:
def get_neighbors_by_embedding(feature_view: str, embedding: list[float], k: int):
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": PUBLIC_ENDPOINT}
    )
    query = NearestNeighborQuery(
        embedding=NearestNeighborQuery.Embedding(value=embedding), neighbor_count=k
    )
    request = SearchNearestEntitiesRequest(feature_view=feature_view, query=query)
    return data_client.search_nearest_entities(request=request)

In [40]:
neighbors = get_neighbors_by_embedding(feature_view, [0.1] * embedding_dim, k=3)
print(neighbors)

nearest_neighbors {
  neighbors {
    entity_id: "3"
    distance: -0.20326116681098938
  }
  neighbors {
    entity_id: "27"
    distance: -0.19727082550525665
  }
  neighbors {
    entity_id: "2"
    distance: -0.19693556427955627
  }
}

